In [25]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import statistics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import root_mean_squared_error
from scripts import dataset

In [26]:
original_data = pd.read_json("../kb/KBR.json")

dimensions = original_data.dimension.unique()
models = original_data.model.unique()
datasets = original_data.name.unique()

datasets_fd = ["BachChoralHarmony", "bank", "cancer", "mushrooms", "soybean"]

data_copy = dataset.get_dataset()

In [27]:
### controllo sulla varianza degli score dei vicini
### la varianza dei "vicini" non è mai superiore a 0.07 su una scala di impatto tra 0 e 1
### inoltre, la varianza è superiore a 0.02 nel 18% dei casi, e superiore a 0.05 nel 3% dei casi
#   superiore a 0.02 -> 0.18622222222222223
#   superiore a 0.03 -> 0.11511111111111111
#   superiore a 0.04 -> 0.06755555555555555
#   superiore a 0.05 -> 0.03688888888888889
#   superiore a 0.06 -> 0.007555555555555556

In [28]:
total = len(datasets)*len(models)*len(dimensions)*5
for th in [0.02, 0.03, 0.04, 0.05, 0.06, 0.07]:
    count = 0
    for dataset in datasets:
        for model in models:
            for dimension in dimensions:

                data = data_copy.copy()

                if dimension == "consistency" and (dataset in datasets_fd):

                        df = data[(data["model"] == model) & (data["dimension"] == dimension) & ((data["name"] == "BachChoralHarmony") | (data["name"] == "mushrooms") | (data["name"] == "bank") | (data["name"] == "cancer") | (data["name"] == "soybean"))].copy()

                        train = df[df["name"] != dataset]
                        test = df[df["name"] == dataset]

                        columns = df.columns
                        features = columns.drop(["name","dimension","model","score","impact","p_correlated_features_0.5","p_correlated_features_0.6","p_correlated_features_0.7","p_correlated_features_0.8","p_correlated_features_0.9"])

                        X_train = train[features]
                        y_train = train["impact"]
                        X_test = test[features]
                        y_test = test["impact"]

                        X_train = StandardScaler().fit_transform(X_train)
                        X_train = np.nan_to_num(X_train)

                        X_test = StandardScaler().fit_transform(X_test)
                        X_test = np.nan_to_num(X_test)

                        knn = KNeighborsRegressor(n_neighbors=14, metric='manhattan')
                        knn.fit(X_train, y_train)
                        y_pred = knn.predict(X_test)
                        error = root_mean_squared_error(y_test, y_pred)
                        #print(dataset+": "+str(error))

                        ### distances
                        distances = knn.kneighbors(X_test, n_neighbors=14, return_distance=False)
                        distances = pd.DataFrame(distances)

                        ### list of datasets names
                        names = df[df["name"] != dataset].copy()
                        names = names.reset_index(drop=True)
                        names["name"] = names["name"] + "_" + names["perc"].astype(str)
                        names = names.reset_index()

                        all_names = df.copy()
                        all_names = all_names.reset_index(drop=True)
                        all_names["name"] = all_names["name"] + "_" + all_names["perc"].astype(str)
                        all_names = all_names.reset_index()

                        for i in range(0,len(names)):
                            distances = distances.replace(names["index"][i],names["name"][i])

                        similar = []
                        for i in range(0,len(distances)):
                            for j in range(0,14):
                                similar.append(distances.loc[i][j])

                            impacts = []
                            for sim in similar:

                                impact = all_names[((all_names.name == sim))].impact
                                impacts.append(impact.values[0])

                            if statistics.variance(impacts) > th:
                                #print(dataset+": "+str(error))
                                #print(statistics.variance(impacts))
                                count = count + 1

                            impacts = []

                elif dimension != "consistency":

                    df = data[(data["model"] == model) & (data["dimension"] == dimension)].copy()

                    train = df[df["name"] != dataset]
                    test = df[df["name"] == dataset]

                    columns = df.columns
                    features = columns.drop(["name","dimension","model","score","impact","p_correlated_features_0.5","p_correlated_features_0.6","p_correlated_features_0.7","p_correlated_features_0.8","p_correlated_features_0.9"])

                    X_train = train[features]
                    y_train = train["impact"]
                    X_test = test[features]
                    y_test = test["impact"]

                    X_train = StandardScaler().fit_transform(X_train)
                    X_train = np.nan_to_num(X_train)

                    X_test = StandardScaler().fit_transform(X_test)
                    X_test = np.nan_to_num(X_test)

                    knn = KNeighborsRegressor(n_neighbors=14, metric='manhattan')
                    knn.fit(X_train, y_train)
                    y_pred = knn.predict(X_test)
                    error = root_mean_squared_error(y_test, y_pred)
                    #print(dataset+": "+str(error))

                    ### distances
                    distances = knn.kneighbors(X_test, n_neighbors=14, return_distance=False)
                    distances = pd.DataFrame(distances)

                    ### list of datasets names
                    names = df[df["name"] != dataset].copy()
                    names = names.reset_index(drop=True)
                    names["name"] = names["name"] + "_" + names["perc"].astype(str)
                    names = names.reset_index()

                    all_names = df.copy()
                    all_names = all_names.reset_index(drop=True)
                    all_names["name"] = all_names["name"] + "_" + all_names["perc"].astype(str)
                    all_names = all_names.reset_index()

                    for i in range(0,len(names)):
                        distances = distances.replace(names["index"][i],names["name"][i])

                    similar = []
                    for i in range(0,len(distances)):
                        for j in range(0,14):
                            similar.append(distances.loc[i][j])

                        impacts = []
                        for sim in similar:

                            impact = all_names[((all_names.name == sim))].impact
                            impacts.append(impact.values[0])

                        if statistics.variance(impacts) > th:
                            #print(dataset+": "+str(error))
                            #print(statistics.variance(impacts))
                            count = count + 1

                        impacts = []
    print(th)
    print(count)
    print(count/total)

0.02
419
0.18622222222222223
0.03
259
0.11511111111111111
0.04
152
0.06755555555555555
0.05
83
0.03688888888888889
0.06
17
0.007555555555555556
0.07
0
0.0


In [29]:
### controllo sulla distanza dei vicini
### studio sulla varianza della distanza tra vicini

In [30]:
variance = []
for dataset in datasets:
    for model in models:
        for dimension in dimensions:

            data = data_copy.copy()

            if dimension == "consistency" and (dataset in datasets_fd):

                    df = data[(data["model"] == model) & (data["dimension"] == dimension) & ((data["name"] == "BachChoralHarmony") | (data["name"] == "mushrooms") | (data["name"] == "bank") | (data["name"] == "cancer") | (data["name"] == "soybean"))].copy()

                    train = df[df["name"] != dataset]
                    test = df[df["name"] == dataset]

                    columns = df.columns
                    features = columns.drop(["name","dimension","model","score","impact","p_correlated_features_0.5","p_correlated_features_0.6","p_correlated_features_0.7","p_correlated_features_0.8","p_correlated_features_0.9"])

                    X_train = train[features]
                    y_train = train["impact"]
                    X_test = test[features]
                    y_test = test["impact"]

                    X_train = StandardScaler().fit_transform(X_train)
                    X_train = np.nan_to_num(X_train)

                    X_test = StandardScaler().fit_transform(X_test)
                    X_test = np.nan_to_num(X_test)

                    knn = KNeighborsRegressor(n_neighbors=14, metric='manhattan')
                    knn.fit(X_train, y_train)
                    y_pred = knn.predict(X_test)
                    error = root_mean_squared_error(y_test, y_pred)
                    #print(dataset+": "+str(error))

                    ### distances
                    distances = knn.kneighbors(X_test, n_neighbors=14, return_distance=True)
                    #distances = pd.DataFrame(distances)

                    #print(dataset)
                    #print(model)
                    #print(dimension)
                    for i in range(0,len(distances[0])):
                        #print("min "+str(distances[0][i].min()))
                        #print("max "+str(distances[0][i].max()))
                        #print("var "+str(distances[0][i].var()))
                        variance.append([dataset, model, dimension, distances[0][i].min(), distances[0][i].max(), distances[0][i].var()])


            elif dimension != "consistency":

                df = data[(data["model"] == model) & (data["dimension"] == dimension)].copy()

                train = df[df["name"] != dataset]
                test = df[df["name"] == dataset]

                columns = df.columns
                features = columns.drop(["name","dimension","model","score","impact","p_correlated_features_0.5","p_correlated_features_0.6","p_correlated_features_0.7","p_correlated_features_0.8","p_correlated_features_0.9"])

                X_train = train[features]
                y_train = train["impact"]
                X_test = test[features]
                y_test = test["impact"]

                X_train = StandardScaler().fit_transform(X_train)
                X_train = np.nan_to_num(X_train)

                X_test = StandardScaler().fit_transform(X_test)
                X_test = np.nan_to_num(X_test)

                knn = KNeighborsRegressor(n_neighbors=14, metric='manhattan')
                knn.fit(X_train, y_train)
                y_pred = knn.predict(X_test)
                error = root_mean_squared_error(y_test, y_pred)
                #print(dataset+": "+str(error))

                ### distances
                distances = knn.kneighbors(X_test, n_neighbors=14, return_distance=True)
                #distances = pd.DataFrame(distances)

                #print(dataset)
                #print(model)
                #print(dimension)
                for i in range(0,len(distances[0])):
                    #print("min "+str(distances[0][i].min()))
                    #print("max "+str(distances[0][i].max()))
                    #print("var "+str(distances[0][i].var()))
                    variance.append([dataset, model, dimension, distances[0][i].min(), distances[0][i].max(), distances[0][i].var()])


In [31]:
variance = pd.DataFrame(variance)

In [32]:
variance.columns = ["dataset","model","dim","min","max","var"]

In [33]:
### la varianza delle distanze è in media 10, oscilla tra 0.4 e 181, ma il 75% delle varianze delle distanze è inferiore a 7.8, tenendo conto che le distanze variano tra 18.7 e 80
### direi che la distanza è abbastanza variante -- adesso proviamo con la cosine distance per farci un'idea di quanto sono vicini/distanti

In [34]:
variance

,dataset,model,dim,min,max,var
0,abalone,DecisionTree,completeness,36.679245,47.233663,15.486477
1,abalone,DecisionTree,completeness,39.212465,43.212252,1.688836
2,abalone,DecisionTree,completeness,31.665653,35.261860,1.141792
3,abalone,DecisionTree,completeness,40.611252,44.682739,1.102592
4,abalone,DecisionTree,completeness,38.703744,46.145853,4.126130
...,...,...,...,...,...,...
1645,wall-robot-navigation,SVC,accuracy,42.510716,51.103526,6.648753
1646,wall-robot-navigation,SVC,accuracy,26.761741,33.887684,5.513065
1647,wall-robot-navigation,SVC,accuracy,24.318314,29.684561,1.863979
1648,wall-robot-navigation,SVC,accuracy,24.055611,31.892165,6.001211


In [35]:
variance.describe()

,min,max,var
count,1650.000000,1650.000000,1650.000000
mean,34.607057,42.622860,10.175873
std,9.425985,11.539102,20.544745
min,18.694570,24.335618,0.477754
25%,27.402358,33.353245,2.256630
50%,32.442520,39.975037,4.249278
75%,39.792358,49.852481,7.849047
max,66.165040,79.156666,181.283421


In [36]:
variance = []
for dataset in datasets:
    for model in models:
        for dimension in dimensions:

            data = data_copy.copy()

            if dimension == "consistency" and (dataset in datasets_fd):

                    df = data[(data["model"] == model) & (data["dimension"] == dimension) & ((data["name"] == "BachChoralHarmony") | (data["name"] == "mushrooms") | (data["name"] == "bank") | (data["name"] == "cancer") | (data["name"] == "soybean"))].copy()

                    train = df[df["name"] != dataset]
                    test = df[df["name"] == dataset]

                    columns = df.columns
                    features = columns.drop(["name","dimension","model","score","impact","p_correlated_features_0.5","p_correlated_features_0.6","p_correlated_features_0.7","p_correlated_features_0.8","p_correlated_features_0.9"])

                    X_train = train[features]
                    y_train = train["impact"]
                    X_test = test[features]
                    y_test = test["impact"]

                    X_train = StandardScaler().fit_transform(X_train)
                    X_train = np.nan_to_num(X_train)

                    X_test = StandardScaler().fit_transform(X_test)
                    X_test = np.nan_to_num(X_test)

                    knn = KNeighborsRegressor(n_neighbors=14, metric='cosine')
                    knn.fit(X_train, y_train)
                    y_pred = knn.predict(X_test)
                    error = root_mean_squared_error(y_test, y_pred)
                    #print(dataset+": "+str(error))

                    ### distances
                    distances = knn.kneighbors(X_test, n_neighbors=14, return_distance=True)
                    #distances = pd.DataFrame(distances)

                    #print(dataset)
                    #print(model)
                    #print(dimension)
                    for i in range(0,len(distances[0])):
                        #print("min "+str(distances[0][i].min()))
                        #print("max "+str(distances[0][i].max()))
                        #print("var "+str(distances[0][i].var()))
                        variance.append([dataset, model, dimension, distances[0][i].min(), distances[0][i].max(), distances[0][i].var()])


            elif dimension != "consistency":

                df = data[(data["model"] == model) & (data["dimension"] == dimension)].copy()

                train = df[df["name"] != dataset]
                test = df[df["name"] == dataset]

                columns = df.columns
                features = columns.drop(["name","dimension","model","score","impact","p_correlated_features_0.5","p_correlated_features_0.6","p_correlated_features_0.7","p_correlated_features_0.8","p_correlated_features_0.9"])

                X_train = train[features]
                y_train = train["impact"]
                X_test = test[features]
                y_test = test["impact"]

                X_train = StandardScaler().fit_transform(X_train)
                X_train = np.nan_to_num(X_train)

                X_test = StandardScaler().fit_transform(X_test)
                X_test = np.nan_to_num(X_test)

                knn = KNeighborsRegressor(n_neighbors=14, metric='cosine')
                knn.fit(X_train, y_train)
                y_pred = knn.predict(X_test)
                error = root_mean_squared_error(y_test, y_pred)
                #print(dataset+": "+str(error))

                ### distances
                distances = knn.kneighbors(X_test, n_neighbors=14, return_distance=True)
                #distances = pd.DataFrame(distances)

                #print(dataset)
                #print(model)
                #print(dimension)
                for i in range(0,len(distances[0])):
                    #print("min "+str(distances[0][i].min()))
                    #print("max "+str(distances[0][i].max()))
                    #print("var "+str(distances[0][i].var()))
                    variance.append([dataset, model, dimension, distances[0][i].min(), distances[0][i].max(), distances[0][i].var()])


In [37]:
variance = pd.DataFrame(variance)
variance.columns = ["dataset", "model", "dim", "min", "max", "var"]
variance

,dataset,model,dim,min,max,var
0,abalone,DecisionTree,completeness,0.570701,0.670549,0.000938
1,abalone,DecisionTree,completeness,0.661850,0.746765,0.000711
2,abalone,DecisionTree,completeness,0.583624,0.804369,0.005750
3,abalone,DecisionTree,completeness,0.487606,0.788665,0.009976
4,abalone,DecisionTree,completeness,0.590355,0.728730,0.001530
...,...,...,...,...,...,...
1645,wall-robot-navigation,SVC,accuracy,0.324261,0.771650,0.023884
1646,wall-robot-navigation,SVC,accuracy,0.547096,0.719327,0.002004
1647,wall-robot-navigation,SVC,accuracy,0.831363,0.858046,0.000058
1648,wall-robot-navigation,SVC,accuracy,0.473550,0.727864,0.007164


In [38]:
variance.describe()

,min,max,var
count,1650.000000,1650.000000,1.650000e+03
mean,0.581191,0.796119,5.872621e-03
std,0.125917,0.103145,6.302457e-03
min,0.245217,0.638208,7.923826e-33
25%,0.491147,0.729437,1.742012e-03
50%,0.581122,0.775277,3.758877e-03
75%,0.671192,0.825691,7.811235e-03
max,1.000000,1.157312,3.878080e-02


In [39]:
### tenendo conto che la cosine distance varia tra 0 (identical), 1 (no correlation), e 2 (totally different)
### la varianza è veramente poca considerato che varia tra 0 e 2 ho una varianza media di 0.00587
### al massimo la varianza è di 0.03 sempre variando tra 0 e 2
### il 75% dei casi ha una variaza minore di 0.007
### i vicini sono sempre molto vicini